<a href="https://colab.research.google.com/github/Celestin25/translation_model/blob/main/Translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from nltk.translate.bleu_score import corpus_bleu

Load the dataset


In [4]:
data_path = '/content/dataset (1).json'
df = pd.read_json(data_path)

Data preprocessing

In [7]:
english_texts = df['english'].tolist()
kinyarwanda_texts = df['kinyarwanda'].tolist()

Tokenization


In [24]:
tokenizer_en = Tokenizer(oov_token='<OOV>', filters='')
tokenizer_en.fit_on_texts(english_texts)
tokenizer_en.word_index['<start>'] = len(tokenizer_en.word_index) + 1
tokenizer_en.word_index['<end>'] = len(tokenizer_en.word_index) + 1

tokenizer_rw = Tokenizer(oov_token='<OOV>', filters='')
tokenizer_rw.fit_on_texts(kinyarwanda_texts)
tokenizer_rw.word_index['<start>'] = len(tokenizer_rw.word_index) + 1
tokenizer_rw.word_index['<end>'] = len(tokenizer_rw.word_index) + 1

Vocaburary sizes

In [30]:
en_vocab_size = len(tokenizer_en.word_index) + 1
rw_vocab_size = len(tokenizer_rw.word_index) + 1

Convert text sequences

In [31]:
encoder_input_data = tokenizer_en.texts_to_sequences(english_texts)
decoder_input_data = tokenizer_rw.texts_to_sequences(kinyarwanda_texts)

Tokens

In [33]:
decoder_target_data = [seq + [tokenizer_rw.word_index['<end>']] for seq in decoder_input_data]
decoder_input_data = [[tokenizer_rw.word_index['<start>']] + seq for seq in decoder_input_data]

Pad sequences

In [36]:
max_seq_length = max([len(seq) for seq in encoder_input_data + decoder_input_data])
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_seq_length, padding='post')

 Define model architecture

In [37]:
embedding_dim = 256
latent_dim = 512

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(en_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(rw_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(rw_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Define the model

In [38]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

compile the model

In [39]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

train the model

In [40]:
model.fit([encoder_input_data, decoder_input_data], decoder_input_data, epochs=50, batch_size=64)


Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 4.9657
Epoch 2/50
1/1 [==============================] - 0s 497ms/step - loss: 4.8425
Epoch 3/50
1/1 [==============================] - 1s 524ms/step - loss: 4.6606
Epoch 4/50
1/1 [==============================] - 0s 497ms/step - loss: 4.2537
Epoch 5/50
1/1 [==============================] - 1s 502ms/step - loss: 3.1717
Epoch 6/50
1/1 [==============================] - 0s 491ms/step - loss: 2.2792
Epoch 7/50
1/1 [==============================] - 1s 538ms/step - loss: 2.7750
Epoch 8/50
1/1 [==============================] - 1s 760ms/step - loss: 2.5358
Epoch 9/50
1/1 [==============================] - 1s 794ms/step - loss: 2.1827
Epoch 10/50
1/1 [==============================] - 1s 805ms/step - loss: 2.1703
Epoch 11/50
1/1 [==============================] - 1s 830ms/step - loss: 2.2175
Epoch 12/50
1/1 [==============================] - 1s 785ms/step - loss: 2.1694
Epoch 13/50
1/1 [==============================] - 1

Evaluation function using BLEU score

In [41]:
def evaluate_model(input_texts, target_texts, model, tokenizer, max_seq_length):
    actual, predicted = [], []
    for i in range(len(input_texts)):
        input_seq = tokenizer.texts_to_sequences([input_texts[i]])[0]
        input_seq = pad_sequences([input_seq], maxlen=max_seq_length, padding='post')
        decoded_sentence = decode_sequence(input_seq, model, tokenizer, max_seq_length)
        actual.append([target_texts[i].split()])
        predicted.append(decoded_sentence.split())
    return corpus_bleu(actual, predicted)

 Decode function for generating translations

In [42]:
def decode_sequence(input_seq, model, tokenizer, max_seq_length):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tokenizer.index_word.get(sampled_token_index, '<OOV>')
        if sampled_token != '<end>':
            decoded_sentence += ' ' + sampled_token
        if (sampled_token == '<end>' or len(decoded_sentence.split()) > max_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

encoder and decoder models

In [43]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

Evaluate the model

In [44]:
bleu_score = evaluate_model(english_texts, kinyarwanda_texts, model, tokenizer_rw, max_seq_length)
print(f'BLEU Score: {bleu_score}')


1/1 [==============================] - 0s 27ms/step
BLEU Score: 6.66832382604621e-232


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_